In [12]:
import pandas as pd
import numpy as np
import simpy
import random

In [ ]:
class Unidad:
    def __init__(self, env, nombre, num_camas, personal, num_kits_medicos):
        self.env = env
        self.nombre = nombre
        self.num_camas = simpy.Resource(env, capacity=num_camas)
        self.personal = simpy.Resource(env, capacity=personal)
        self.num_kits_medicos = simpy.Resource(env, capacity=num_kits_medicos)


In [14]:
class Hospital:
    def __init__(self, env, recepcionistas, unidades):
        self.env = env
        self.unidades = unidades
        self.recepcionistas = simpy.Resource(env, capacity=recepcionistas)

In [27]:
class Paciente:
    def __init__(self, env, id, hospital: Hospital, unidad: Unidad):
        self.env = env
        self.id = id
        self.hospital = hospital
        self.unidad = unidad
        self.cirugia = random.choice([True, False]) if unidad.nombre == "Urgencias" else False
        self.alcanzo_cama = False
        
        self.env.process(self.start())
        self.eventos = pd.DataFrame()
    
    def start(self):
        print(f"{self.env.now}: Paciente {self.id} llega al hospital a la unidad {self.unidad.nombre}.")

        with self.hospital.recepcionistas.request() as request:
            yield request
            print(f"{self.env.now}: Paciente {self.id} es atendido.")
            self.eventos = pd.concat([self.eventos, pd.DataFrame({
                "id": [self.id],
                "evento": ["LLEGADA"],
                "time": [self.env.now]
            })])

            if self.unidad.nombre == "Urgencias" or self.unidad.nombre == "UCI":
                tiempo_atencion_inferior = 1
                tiempo_atencion_superior = 7
                cant_personal = random.randint(3, 8)
            else:
                tiempo_atencion_inferior = 7
                tiempo_atencion_superior = 25
                cant_personal = 1 #Solo se necesita un doctor para una consulta general

            # El paciente toma una cama y se le asigna personal
            with self.unidad.num_camas.request() as cama_request:
                yield cama_request
                with self.unidad.personal.request() as personal_request:
                    yield personal_request
                    yield self.env.timeout(random.randint(tiempo_atencion_inferior, tiempo_atencion_superior))
                    print(f"{self.env.now}: Al Paciente {self.id} se le asigno una cama y {cant_personal} miembro(s) del personal.")
                    self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": ["ASIGNACION_CAMA_PERSONAL"],
                        "time": [self.env.now]
                    })])

            #Si el paciente es de urgencias y necesita cirugia, espera que se le asigne un kit medico
            if self.cirugia:
                print(f"{self.env.now}: El Paciente {self.id} necesita cirugia.")
                with self.unidad.num_kits_medicos.request() as kit_request:
                    yield kit_request
                    tiempo_espera_kit = random.randint(15, 60*8) #Se consideran cirugias de 8 horas maximo
                    yield self.env.timeout(tiempo_espera_kit)
                    
                    if tiempo_espera_kit > 60:
                        print(f"{self.env.now}: El Paciente {self.id} murio esperando un kit medico.")
                        self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": ["MUERTE"],
                        "time": [self.env.now]
                    })])
                    else:
                        print(f"{self.env.now}: Al Paciente {self.id} se le asigno un kit medico.")
                        self.eventos = pd.concat([self.eventos, pd.DataFrame({
                        "id": [self.id],
                        "evento": ["ASIGNACION_KIT"],
                        "time": [self.env.now]
                    })])



In [16]:
class Simulation:
    def __init__(self, env, unidades_settings, recepcionistas):
        self.env = env
        self.unidades = [Unidad(env, u["nombre"], u["num_camas"], u["personal"], u["num_kits_medicos"]) for u in unidades_settings]
        self.hospital = Hospital(env, recepcionistas, self.unidades)
        self.pacientes = []

        self.env.process(self.run_simulation())
    
    def run_simulation(self):
        while True:
            # Generar de 1 a 5 pacientes a la vez
            yield self.env.timeout(random.randint(1, 5))

            for _ in range(random.randint(1, 5)):
                paciente = Paciente(
                    self.env,
                    len(self.pacientes) + 1,
                    self.hospital,
                    random.choice(self.unidades)
                )
                self.pacientes.append(paciente)

# Inicio de la simulacion

In [28]:
env = simpy.Environment()
unidades_settings = [
    {"nombre": "Urgencias", "num_camas": 20, "personal": 30, "num_kits_medicos": 8},
    {"nombre": "UCI", "num_camas": 35, "personal": 25, "num_kits_medicos": 1},
    {"nombre": "Consultorios", "num_camas": 10, "personal": 8, "num_kits_medicos": 1}
]
num_recepcionistas = 2

#random.seed(111)

simulacion = Simulation(env, unidades_settings, num_recepcionistas)
env.run(until=60*8)

4: Paciente 1 llega al hospital a la unidad Consultorios.
4: Paciente 2 llega al hospital a la unidad Consultorios.
4: Paciente 1 es atendido.
4: Paciente 2 es atendido.
6: Paciente 3 llega al hospital a la unidad UCI.
6: Paciente 4 llega al hospital a la unidad Urgencias.
7: Paciente 5 llega al hospital a la unidad UCI.
7: Paciente 6 llega al hospital a la unidad Consultorios.
9: Paciente 7 llega al hospital a la unidad Urgencias.
9: Paciente 8 llega al hospital a la unidad Urgencias.
12: Paciente 9 llega al hospital a la unidad Consultorios.
12: Paciente 10 llega al hospital a la unidad UCI.
12: Paciente 11 llega al hospital a la unidad UCI.
12: Paciente 12 llega al hospital a la unidad Urgencias.
12: Paciente 13 llega al hospital a la unidad Consultorios.
14: Al Paciente 1 se le asigno una cama y 1 miembro(s) del personal.
14: Paciente 14 llega al hospital a la unidad Urgencias.
14: Paciente 3 es atendido.
16: Al Paciente 3 se le asigno una cama y 3 miembro(s) del personal.
16: Paci